In [5]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage import exposure
from load_images import load_images_from_zip
import zipfile

In [6]:
# Load images and labels
dataset_path = 'images.zip'
df = load_images_from_zip(dataset_path)

In [7]:
label_encoder = LabelEncoder()
df['labels_encoded'] = label_encoder.fit_transform(df['labels'])
num_classes = len(label_encoder.classes_)

In [8]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
# Function to extract HOG features from an image
def extract_hog_features(img):
    img = img.convert('L')  # Convert to grayscale
    img = img.resize((64, 64))  # Resize for consistency
    fd, hog_image = hog(np.array(img), orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=True)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd

In [10]:
# Extract HOG features for training set
train_features = []
for index, row in train_df.iterrows():
    with zipfile.ZipFile('images.zip', 'r') as zip_ref:
        with zip_ref.open(row['image_path']) as file:
            img = Image.open(file)
            hog_features = extract_hog_features(img)
            train_features.append(hog_features)
train_labels = train_df['labels_encoded']

KeyboardInterrupt: 

In [ ]:
# Extract HOG features for test set
test_features = []
for index, row in test_df.iterrows():
    with zipfile.ZipFile('images.zip', 'r') as zip_ref:
        with zip_ref.open(row['image_path']) as file:
            img = Image.open(file)
            hog_features = extract_hog_features(img)
            test_features.append(hog_features)
test_labels = test_df['labels_encoded']

In [ ]:
# Standardize the features
scaler = StandardScaler()
train_features_std = scaler.fit_transform(train_features)
test_features_std = scaler.transform(test_features)

In [ ]:
# Train an SVM model
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(train_features_std, train_labels)

In [ ]:
# Make predictions on the test set
svm_predictions = svm_model.predict(test_features_std)

In [ ]:
# Evaluate the SVM model
accuracy = accuracy_score(test_labels, svm_predictions)
print(f'SVM Accuracy: {accuracy}')
